In [1]:
import pickle
import math
import numpy as np
import time
from collections import defaultdict

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.neural_network import MLPRegressor

In [2]:
AutoML_path = ""
log_folder = "alphad3m_log"

log_DDN = AutoML_path+log_folder+'/DDN_single_time'
log_pipeline_steps = AutoML_path+log_folder+'/pipeline_steps_class_02272022'
log_uniq_prim = AutoML_path+log_folder+'/uniq_prim_02272022'

combination_name = 'DDN+BO'
today = '03102022'
log_target_test_pred_time = AutoML_path+log_folder+'/'+combination_name+'_target_test_pred_time_'+today


In [3]:
DDN = pickle.load(open(log_DDN, "rb"))
print(len(DDN))

pipeline_steps_class = pickle.load(open(log_pipeline_steps, 'rb'))
print(len(pipeline_steps_class))


39
7811


In [4]:
from sklearn.gaussian_process.kernels import Matern
from sklearn.gaussian_process import GaussianProcessRegressor

In [9]:
target_test_pred_time = {}
data_names = list(DDN.keys())
data_embed = DDN

for data_index, data_name in enumerate(data_names):
    print("="*40)
    print(data_index, data_name)
    
    task_test = [data_name]
    candidate_score = DDN[data_name]
    prev_key = None
    dict_key = None
    len_task_train = 0
    prev_len_task_train = 0
    for threshold in np.arange(0.05, 0.5, 0.05):
        prev_key = dict_key
        dict_key = '_'.join([data_name, combination_name, str(round(threshold,2))])
        
        start = time.time()
        task_train = [i for i in data_names if i!=data_name and abs(DDN[i]-candidate_score)<threshold]
        prev_len_task_train = len_task_train
        len_task_train = len(task_train)
        if len_task_train == 0:
            target_test_pred_time[dict_key] = 'N.A.'
            pickle.dump(target_test_pred_time, open(log_target_test_pred_time, "wb"))
            continue
            
        if len_task_train == prev_len_task_train:
            print(prev_key, dict_key)
            print(prev_len_task_train, len_task_train)
            target_test_pred_time[dict_key] =  target_test_pred_time[prev_key]
            continue

        X_train = np.array([np.hstack((data_embed[i[0]], i[1])) for i in pipeline_steps_class if i[0] in task_train and i[3] != "Timeout" and not np.isinf(i[3])])
        y_train = np.array([i[3] for i in pipeline_steps_class if i[0] in task_train and i[3] != "Timeout" and not np.isinf(i[3])])
        X_test = np.array([np.hstack((data_embed[i[0]], i[1])) for i in pipeline_steps_class if i[0] in task_test and i[3] != "Timeout" and not np.isinf(i[3])])
        y_test = np.array([i[3] for i in pipeline_steps_class if i[0] in task_test and i[3] != "Timeout" and not np.isinf(i[3])])
        print("X_train shape:", X_train.shape, "y_train shape:", y_train.shape)
        print("X_test shape:", X_test.shape, "y_test shape:", y_test.shape)

        model = GaussianProcessRegressor(
            kernel=Matern(nu=2.5),alpha=1e-6,normalize_y=True,
            n_restarts_optimizer=0,random_state=1
        )
        
        import warnings
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            model.fit(X_train, y_train)
        end = time.time()
        duration = end-start

        y_pred = model.predict(X_test)
        mae = mean_absolute_error(y_test, y_pred)
        mse = mean_squared_error(y_test, y_pred)
        print("Threshold", threshold, "MAE:", mae, "MSE:", mse)
        print("Time:", duration)
        target_test_pred_time[dict_key] = (y_test, y_pred, duration, mae, mse)
        pickle.dump(target_test_pred_time, open(log_target_test_pred_time, 'wb'))
    


0 adult
X_train shape: (1526, 37) y_train shape: (1526,)
X_test shape: (452, 37) y_test shape: (452,)
Threshold 0.05 MAE: 0.06972411392660854 MSE: 0.02031647724703847
Time: 0.5723600387573242
X_train shape: (3405, 37) y_train shape: (3405,)
X_test shape: (452, 37) y_test shape: (452,)
Threshold 0.1 MAE: 0.06889635764205287 MSE: 0.02015159877372517
Time: 3.8732473850250244
X_train shape: (5415, 37) y_train shape: (5415,)
X_test shape: (452, 37) y_test shape: (452,)
Threshold 0.15000000000000002 MAE: 0.07354906064871104 MSE: 0.018370969061430337
Time: 14.89076280593872
X_train shape: (5892, 37) y_train shape: (5892,)
X_test shape: (452, 37) y_test shape: (452,)
Threshold 0.2 MAE: 0.06742265699158145 MSE: 0.018840165979025002
Time: 20.001418113708496
X_train shape: (6745, 37) y_train shape: (6745,)
X_test shape: (452, 37) y_test shape: (452,)
Threshold 0.25 MAE: 0.08663327498346372 MSE: 0.018117160395707873
Time: 26.533244371414185
X_train shape: (6931, 37) y_train shape: (6931,)
X_test s

X_test shape: (216, 37) y_test shape: (216,)
Threshold 0.15000000000000002 MAE: 0.0893868272928998 MSE: 0.009590119070502364
Time: 17.219241857528687
X_train shape: (6128, 37) y_train shape: (6128,)
X_test shape: (216, 37) y_test shape: (216,)
Threshold 0.2 MAE: 0.10971711153120675 MSE: 0.01305195260024042
Time: 22.694273948669434
X_train shape: (6981, 37) y_train shape: (6981,)
X_test shape: (216, 37) y_test shape: (216,)
Threshold 0.25 MAE: 0.13498690154282178 MSE: 0.018882954120427738
Time: 30.04792618751526
X_train shape: (7167, 37) y_train shape: (7167,)
X_test shape: (216, 37) y_test shape: (216,)
Threshold 0.3 MAE: 0.13860799083276923 MSE: 0.01984768078312371
Time: 32.16665482521057
Australian_DDN+BO_0.3 Australian_DDN+BO_0.35
35 35
X_train shape: (7383, 37) y_train shape: (7383,)
X_test shape: (216, 37) y_test shape: (216,)
Threshold 0.4 MAE: 0.1436480457575848 MSE: 0.02124807058352602
Time: 31.363654851913452
Australian_DDN+BO_0.4 Australian_DDN+BO_0.45
36 36
6 bank-marketing


Threshold 0.3 MAE: 0.15458539146685732 MSE: 0.03781551838298862
Time: 25.67836284637451
X_train shape: (7006, 37) y_train shape: (7006,)
X_test shape: (196, 37) y_test shape: (196,)
Threshold 0.35000000000000003 MAE: 0.15737755731790035 MSE: 0.03840560698360381
Time: 29.450445890426636
X_train shape: (7187, 37) y_train shape: (7187,)
X_test shape: (196, 37) y_test shape: (196,)
Threshold 0.4 MAE: 0.16013217959792223 MSE: 0.03901959589634166
Time: 32.73251795768738
X_train shape: (7403, 37) y_train shape: (7403,)
X_test shape: (196, 37) y_test shape: (196,)
Threshold 0.45 MAE: 0.16405120281703867 MSE: 0.039950277175857495
Time: 33.27726984024048
11 connect-4
X_train shape: (1694, 37) y_train shape: (1694,)
X_test shape: (216, 37) y_test shape: (216,)
Threshold 0.05 MAE: 0.10822262010332873 MSE: 0.028072571685747862
Time: 0.8819999694824219
X_train shape: (2547, 37) y_train shape: (2547,)
X_test shape: (216, 37) y_test shape: (216,)
Threshold 0.1 MAE: 0.13061193997923332 MSE: 0.029012824

Threshold 0.35000000000000003 MAE: 0.5657887344123221 MSE: 0.3351886145224994
Time: 29.722817182540894
dionis_DDN+BO_0.35 dionis_DDN+BO_0.4
37 37
X_train shape: (7607, 37) y_train shape: (7607,)
X_test shape: (204, 37) y_test shape: (204,)
Threshold 0.45 MAE: 0.550723018384964 MSE: 0.3183675655136474
Time: 32.79226088523865
16 fabert
X_train shape: (665, 37) y_train shape: (665,)
X_test shape: (188, 37) y_test shape: (188,)
Threshold 0.05 MAE: 0.13307824901861848 MSE: 0.02605595348205156
Time: 0.15297794342041016
X_train shape: (1919, 37) y_train shape: (1919,)
X_test shape: (188, 37) y_test shape: (188,)
Threshold 0.1 MAE: 0.13195503230441924 MSE: 0.02575826367065046
Time: 1.229133129119873
X_train shape: (2977, 37) y_train shape: (2977,)
X_test shape: (188, 37) y_test shape: (188,)
Threshold 0.15000000000000002 MAE: 0.17732499323631876 MSE: 0.03979028634645062
Time: 2.9963388442993164
fabert_DDN+BO_0.15 fabert_DDN+BO_0.2
15 15
X_train shape: (4963, 37) y_train shape: (4963,)
X_test s

Threshold 0.25 MAE: 0.12259871632249092 MSE: 0.020297748636663823
Time: 11.797086954116821
X_train shape: (5756, 37) y_train shape: (5756,)
X_test shape: (216, 37) y_test shape: (216,)
Threshold 0.3 MAE: 0.12259868768675349 MSE: 0.020297741567159185
Time: 16.650169849395752
X_train shape: (6884, 37) y_train shape: (6884,)
X_test shape: (216, 37) y_test shape: (216,)
Threshold 0.35000000000000003 MAE: 0.122598680665346 MSE: 0.020297739833735503
Time: 25.26802897453308
X_train shape: (7595, 37) y_train shape: (7595,)
X_test shape: (216, 37) y_test shape: (216,)
Threshold 0.4 MAE: 0.12259866251879455 MSE: 0.020297735353770642
Time: 33.578943967819214
jannis_DDN+BO_0.4 jannis_DDN+BO_0.45
38 38
22 jasmine
X_train shape: (1274, 37) y_train shape: (1274,)
X_test shape: (216, 37) y_test shape: (216,)
Threshold 0.05 MAE: 0.10467971613439785 MSE: 0.012257571739545448
Time: 0.4640839099884033
X_train shape: (3240, 37) y_train shape: (3240,)
X_test shape: (216, 37) y_test shape: (216,)
Threshold 0

Threshold 0.4 MAE: 0.2091932101716409 MSE: 0.04739652631257709
Time: 26.538382053375244
X_train shape: (7117, 37) y_train shape: (7117,)
X_test shape: (266, 37) y_test shape: (266,)
Threshold 0.45 MAE: 0.21079166197431087 MSE: 0.04824804617304786
Time: 28.686530828475952
27 mfeat-factors
X_train shape: (1203, 37) y_train shape: (1203,)
X_test shape: (216, 37) y_test shape: (216,)
Threshold 0.05 MAE: 0.09253647709430125 MSE: 0.022385090950761372
Time: 0.40268492698669434
X_train shape: (2240, 37) y_train shape: (2240,)
X_test shape: (216, 37) y_test shape: (216,)
Threshold 0.1 MAE: 0.10621330987899275 MSE: 0.022985196439836028
Time: 1.631654977798462
X_train shape: (3822, 37) y_train shape: (3822,)
X_test shape: (216, 37) y_test shape: (216,)
Threshold 0.15000000000000002 MAE: 0.1307252522097194 MSE: 0.026126365665042863
Time: 5.399312973022461
X_train shape: (4218, 37) y_train shape: (4218,)
X_test shape: (216, 37) y_test shape: (216,)
Threshold 0.2 MAE: 0.1351041756507667 MSE: 0.02687

Threshold 0.05 MAE: 0.06397672097092386 MSE: 0.005507088697262857
Time: 0.9649078845977783
X_train shape: (2396, 37) y_train shape: (2396,)
X_test shape: (60, 37) y_test shape: (60,)
Threshold 0.1 MAE: 0.0664500373564995 MSE: 0.005918461383618992
Time: 1.8074049949645996
X_train shape: (4374, 37) y_train shape: (4374,)
X_test shape: (60, 37) y_test shape: (60,)
Threshold 0.15000000000000002 MAE: 0.045128334440335226 MSE: 0.0030802362309454755
Time: 8.438350915908813
riccardo_DDN+BO_0.15 riccardo_DDN+BO_0.2
20 20
X_train shape: (5432, 37) y_train shape: (5432,)
X_test shape: (60, 37) y_test shape: (60,)
Threshold 0.25 MAE: 0.04328820048833907 MSE: 0.003146078888153113
Time: 13.990693092346191
X_train shape: (6284, 37) y_train shape: (6284,)
X_test shape: (60, 37) y_test shape: (60,)
Threshold 0.3 MAE: 0.05723476651637711 MSE: 0.00554221838354652
Time: 21.06878113746643
X_train shape: (6921, 37) y_train shape: (6921,)
X_test shape: (60, 37) y_test shape: (60,)
Threshold 0.350000000000000

Threshold 0.2 MAE: 0.12777449435250854 MSE: 0.023731225436217137
Time: 11.055739164352417
X_train shape: (6612, 37) y_train shape: (6612,)
X_test shape: (216, 37) y_test shape: (216,)
Threshold 0.25 MAE: 0.1290658180347148 MSE: 0.027642695978602035
Time: 20.2863130569458
X_train shape: (7383, 37) y_train shape: (7383,)
X_test shape: (216, 37) y_test shape: (216,)
Threshold 0.3 MAE: 0.1332570044112673 MSE: 0.03102732902512062
Time: 25.721511125564575
X_train shape: (7391, 37) y_train shape: (7391,)
X_test shape: (216, 37) y_test shape: (216,)
Threshold 0.35000000000000003 MAE: 0.13308609549828765 MSE: 0.030921414809125327
Time: 25.773870944976807
vehicle_DDN+BO_0.35 vehicle_DDN+BO_0.4
37 37
X_train shape: (7595, 37) y_train shape: (7595,)
X_test shape: (216, 37) y_test shape: (216,)
Threshold 0.45 MAE: 0.12951930245741733 MSE: 0.028162057429211445
Time: 28.160654067993164
38 volkert
X_train shape: (637, 37) y_train shape: (637,)
X_test shape: (216, 37) y_test shape: (216,)
Threshold 0.0

In [8]:
len(data_names)

39